Given data of customer purchases, find all customers who made at least two purchases within 30 days of their first transaction.
 
Return a DataFrame with:

	customer_id

	first_transaction_date

	repeat_within_30_days (boolean)
 
Example input:
import pandas as pd
data = {
    'customer_id': [1, 1, 1, 2, 2, 3, 4],
    'transaction_date': pd.to_datetime([
      '2023-01-01', '2023-01-10', '2023-03-01',  # customer 1
      '2023-02-01', '2023-04-05',                # customer 2
      '2023-01-01',                              # customer 3
      '2023-05-20'                               # customer 4
    ]),
    'amount_spent': [100, 150, 80, 200, 50, 40, 90]
}

transactions = pd.DataFrame(data)
 
 
Expected output:
	   customer_id first_transaction_date  repeat_within_30_days

	0            1            2023-01-01                   True

	1            2            2023-02-01                  False

	2            3            2023-01-01                  False

	3            4            2023-05-20                  False
 

In [2]:
import pandas as pd

data = {
    'customer_id': [1, 1, 1, 2, 2, 3, 4],
    'transaction_date': pd.to_datetime([
      '2023-01-01', '2023-01-10', '2023-03-01',  # customer 1
      '2023-02-01', '2023-04-05',                # customer 2
      '2023-01-01',                              # customer 3
      '2023-05-20'                               # customer 4
    ]),
    'amount_spent': [100, 150, 80, 200, 50, 40, 90]
}


transactions = pd.DataFrame(data)

In [3]:
transactions

,customer_id,transaction_date,amount_spent
0,1,2023-01-01,100
1,1,2023-01-10,150
2,1,2023-03-01,80
3,2,2023-02-01,200
4,2,2023-04-05,50
5,3,2023-01-01,40
6,4,2023-05-20,90


In [4]:
transactions_sorted = transactions.sort_values(by=['customer_id', 'transaction_date'])

In [5]:
cust_trans_first = dict()
for index, row in transactions_sorted.iterrows():
    cid = row['customer_id']
    trandate = row['transaction_date']
    if cid not in cust_trans_first:
        cust_trans_first[cid] = trandate

In [6]:
cust_trans_first

{1: Timestamp('2023-01-01 00:00:00'),
 2: Timestamp('2023-02-01 00:00:00'),
 3: Timestamp('2023-01-01 00:00:00'),
 4: Timestamp('2023-05-20 00:00:00')}

In [7]:
def get_first(cid):
    return cust_trans_first[cid]

In [8]:
transactions = transactions_sorted

In [9]:
transactions['days_to_first_trans'] = (transactions['transaction_date'] - transactions['customer_id'].apply(get_first)).dt.days
transactions

,customer_id,transaction_date,amount_spent,days_to_first_trans
0,1,2023-01-01,100,0
1,1,2023-01-10,150,9
2,1,2023-03-01,80,59
3,2,2023-02-01,200,0
4,2,2023-04-05,50,63
5,3,2023-01-01,40,0
6,4,2023-05-20,90,0


In [10]:
transactions['within_30'] = transactions['days_to_first_trans'] <= 30
transactions

,customer_id,transaction_date,amount_spent,days_to_first_trans,within_30
0,1,2023-01-01,100,0,True
1,1,2023-01-10,150,9,True
2,1,2023-03-01,80,59,False
3,2,2023-02-01,200,0,True
4,2,2023-04-05,50,63,False
5,3,2023-01-01,40,0,True
6,4,2023-05-20,90,0,True


In [11]:
cust_first_date = dict()
cust_within_30 = dict()

init_data = dict()
for index, row in transactions.iterrows():
    customer_id = row['customer_id']
    transaction_date = row['transaction_date']
    within_30 = row['within_30']
    if customer_id not in cust_first_date:
        cust_first_date[customer_id] = transaction_date

    if customer_id not in cust_within_30:
        cust_within_30[customer_id] = 1 if within_30 else 0
    else:
        cust_within_30[customer_id] += 1 if within_30 else 0
cust_within_30 = {k:v>=2 for k,v in cust_within_30.items()}

In [12]:
res = list()
for cid, cdate in cust_first_date.items():
    res.append({'customer_id': cid, 'first_transaction_date': cdate, 'repeat_within_30_days': cust_within_30[cid]})

pd.DataFrame(res)

,customer_id,first_transaction_date,repeat_within_30_days
0,1,2023-01-01,True
1,2,2023-02-01,False
2,3,2023-01-01,False
3,4,2023-05-20,False


[GPT Answer](https://chatgpt.com/share/68e25608-11e0-8007-bf23-6b1120afe3e7)

step 1: https://chatgpt.com/share/68e449ad-1db0-8007-962d-5bb49d0be7aa

In [13]:
# Step 1: Get the first transaction date for each customer
first_tx = transactions.groupby('customer_id')['transaction_date'].min().reset_index()
first_tx.rename(columns={'transaction_date': 'first_transaction_date'}, inplace=True)

In [14]:
first_tx

,customer_id,first_transaction_date
0,1,2023-01-01
1,2,2023-02-01
2,3,2023-01-01
3,4,2023-05-20
